# Exercise 6.3 Geographic Visualistaions

# 01. Import Libraries and Data

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib 
import os
import folium
import json

In [6]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [119]:
country_geo = r"C:\Users\kthav\My Pc\Desktop\Career Foundry\Sourcing Open Data\Final Project\Data\Original Data\custom.geo.json"

In [12]:
# create path

path = r'C:\Users\kthav\My Pc\Desktop\Career Foundry\Sourcing Open Data\Final Project'

In [14]:
path

'C:\\Users\\kthav\\My Pc\\Desktop\\Career Foundry\\Sourcing Open Data\\Final Project'

In [18]:
# import merged_clean.csv as df

df = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'merged_clean.csv'), index_col = False)

In [20]:
df.shape

(782, 12)

In [22]:
df.head()

Unnamed: 0      Country          Region  Happiness Rank  Happiness Score  \
0           0  Switzerland  Western Europe               1            7.587   
1           1      Iceland  Western Europe               2            7.561   
2           2      Denmark  Western Europe               3            7.527   
3           3       Norway  Western Europe               4            7.522   
4           4       Canada   North America               5            7.427   

   Economy (GDP per Capita)   Family  Health (Life Expectancy)  Freedom  \
0                   1.39651  1.34951                   0.94143  0.66557   
1                   1.30232  1.40223                   0.94784  0.62877   
2                   1.32548  1.36058                   0.87464  0.64938   
3                   1.45900  1.33095                   0.88521  0.66973   
4                   1.32629  1.32261                   0.90563  0.63297   

   Trust (Government Corruption)  Generosity  Year  
0                        0.41978     0.29678  2015  
1                        0.14145     0.43630  2015  
2                        0.48357     0.34139  2015  
3                        0.36503     0.34699  2015  
4                        0.32957     0.45811  2015

In [24]:
df.columns

Index(['Unnamed: 0', 'Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Year'],
      dtype='object')

# 02. Data Wrangling

In [55]:
# select only the necessary columns and put them in a list called columns

columns = ['Country', 'Happiness Rank', 'Year']

In [57]:
# create a subset

df_country = df[columns]

In [59]:
df_country.head()

Country  Happiness Rank  Year
0  Switzerland               1  2015
1      Iceland               2  2015
2      Denmark               3  2015
3       Norway               4  2015
4       Canada               5  2015

In [61]:
df_country.shape

(782, 3)

# 03. Conduct Consistency Checks

In [64]:
# Check for missing values

df_country.isnull().sum()

Country           0
Happiness Rank    0
Year              0
dtype: int64

In [68]:
# check for duplicates

dups = df_country.duplicated()

In [70]:
dups.shape

(782,)

In [72]:
df_country.dtypes

Country           object
Happiness Rank     int64
Year               int64
dtype: object

# 04. Plotting a Choropleth

In [77]:
# create a dataframe with just the data that is needed

# Filter the data for the year 2019
df_2019 = df_country[df_country['Year'] == 2019]

In [83]:
# Sort and select top 10 and bottom 10 countries
top_10 = df_2019.nsmallest(10, 'Happiness Rank')
bottom_10 = df_2019.nlargest(10, 'Happiness Rank')

In [107]:
# Add a column for category
top_10['Category'] = 1
bottom_10['Category'] = 0

In [109]:
# Combine the top and bottom datasets
top_bottom = pd.concat([top_10, bottom_10])

In [111]:
top_bottom.columns

Index(['Country', 'Happiness Rank', 'Year', 'Category'], dtype='object')

In [113]:
top_bottom.head

<bound method NDFrame.head of                       Country  Happiness Rank  Year  Category
626                   Finland               1  2019         1
627                   Denmark               2  2019         1
628                    Norway               3  2019         1
629                   Iceland               4  2019         1
630               Netherlands               5  2019         1
631               Switzerland               6  2019         1
632                    Sweden               7  2019         1
633               New Zealand               8  2019         1
634                    Canada               9  2019         1
635                   Austria              10  2019         1
781               South Sudan             156  2019         0
780  Central African Republic             155  2019         0
779               Afghanistan             154  2019         0
778                  Tanzania             153  2019         0
777                    Rwanda           

In [127]:
# Path to GeoJSON file
country_geo_path = r"C:\Users\kthav\My Pc\Desktop\Career Foundry\Sourcing Open Data\Final Project\Data\Original Data\custom.geo.json"

# Open and parse the GeoJSON file
with open(country_geo_path, encoding='utf-8') as f:
    country_geo = json.load(f)

# Initialize the Folium map
m = folium.Map(location=[0, 0], zoom_start=2)

# Add the choropleth layer
folium.Choropleth(
    geo_data=country_geo,  # Pass the parsed GeoJSON object
    data=top_bottom,       # DataFrame with happiness data
    columns=['Country', 'Category'],  # Columns for choropleth
    key_on='feature.properties.name',  # Match GeoJSON property
    fill_color='RdYlBu',  # Color scheme
    fill_opacity=0.6,
    line_opacity=0.1,
    legend_name='Happiness Scores (Top 10: Blue, Bottom 10: Red)'
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m